# Doc2Vec Model

In [16]:
import re
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from scipy import spatial




In [58]:
#scrapping reddit data and store it in a txt file
def get_reddit_data(nbPost):
    from pmaw import PushshiftAPI
    api = PushshiftAPI()
    comments = api.search_submissions(subreddit="Politiquefrancaise", limit=nbPost, TimeoutError=1000)
    
    comment_list = [comment['selftext'] for comment in comments]
    with open("reddit_data.txt", "w", encoding="utf-8") as f:
        for comment in comment_list:
            #remove urls
            comment = re.sub(r'http\S+', '', comment)
            f.write(comment)
    return 

get_reddit_data(10000)



Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.


In [49]:
#télcharge "quinze essais politiques" dans data et en fait une liste
# opening the file in read mode
#my_file = open("data/quinze_essais_politiques.txt", "r") -> not working
  
f = open("reddit_data.txt", "r", encoding="utf-8")
# reading the file
corpus = f.read()
#remove \n
corpus = corpus.replace('\n', ' ')
corpus = corpus.replace('.', ',')

  
# replacing end splitting the text 
# when newline ('\n') is seen.
corpus = corpus.split(",")
f.close()
corpus[2]

" à l'occasion de la victoire de l'équipe Algérienne en Coupe arabe"

Use re module to preprocess data

Convert all letters into lowercase

Remove punctuations, numbers, etc

In [50]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()
    #remove punctuation
    corpus[i] = re.sub(r'[^\w\s]','',corpus[i])
    #make a list of corpus[i]
    corpus[i] = corpus[i].split()
    

corpus[2]


['à',
 'loccasion',
 'de',
 'la',
 'victoire',
 'de',
 'léquipe',
 'algérienne',
 'en',
 'coupe',
 'arabe']

For the doc2vec model, input data should be in format of iterable TaggedDocuments"

Each TaggedDocument instance comprises words and tags

Hence, each document (i.e., a sentence or paragraph) should have a unique tag which is identifiable

In [51]:
for i in range(len(corpus)):
    corpus[i] = TaggedDocument(words = corpus[i], tags = ['sent{}'.format(i)])    # converting each sentence into a TaggedDocument
corpus[2]

TaggedDocument(words=['à', 'loccasion', 'de', 'la', 'victoire', 'de', 'léquipe', 'algérienne', 'en', 'coupe', 'arabe'], tags=['sent2'])

In [55]:
model = Doc2Vec(documents = corpus, vector_size = 10, min_count = 1)
model.init_sims(replace = True)

model.save('doc2vec_model')
model = Doc2Vec.load('doc2vec_model')


C:\Users\maria\AppData\Local\Temp\ipykernel_14244\2314325498.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace = True)


In [54]:
v1 = model.infer_vector([' front national'])    # in doc2vec, infer_vector() function is used to infer the vector embedding of a document
v2 = model.infer_vector(['insoumis'])    # in doc2vec, infer_vector() function is used to infer the vector embedding of a document
# define a function that computes cosine similarity between two words
def cosine_similarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)
cosine_similarity(v1, v2)






-0.09528177231550217